In [5]:
import os
from xml2mask_v2 import xml2mask_v2
from matplotlib import pyplot as plt
import cv2
from time import time
import numpy as np
from PIL import Image
import pandas as pd
from copy import deepcopy

In [6]:
DL_src = r'\\kukissd\Kyu_Sync\deeplab with ashley\training_set\5x\classification_for_DL_skin_v3'
DL_results = [os.path.join(DL_src,_) for _ in os.listdir(DL_src) if _.endswith('tif')]
# DL_results = [_ for _ in DL_results if not _.endswith(('024.tif','002.tif'))]

svs_src = r'\\kukissd\Kyu_Sync\deeplab with ashley\training_set'
svss = [os.path.join(svs_src,_) for _ in os.listdir(svs_src) if _.endswith('svs')]
# svss = [_ for _ in svss if not _.endswith(('024.svs','002.svs'))]
xmls = [_.replace('.svs','.xml') for _ in svss]

In [7]:
sort_index = [0,1,9,3,2,4,5,6,7,8,10,11]
accuracies = np.zeros((0,len(sort_index)+1))
# columns = ['fn']+list(range(len(sort_index)))
columns = ['filename','corneum','basale','root','follicle','muscle','oil','sweat','nerve','blood','collagen','fat','whitespace']
accuracies = pd.DataFrame(accuracies,columns=columns)
accuracies

,filename,corneum,basale,root,follicle,muscle,oil,sweat,nerve,blood,collagen,fat,whitespace


In [8]:
for idx,(svs_path,xml_path,DL) in enumerate(zip(svss,xmls,DL_results)):
    mask = xml2mask_v2(svs_path,xml_path,mag=5,sort_index=sort_index,binary=False)
    DL_result_img = np.array(Image.open(DL))
    mask = cv2.resize(mask, dsize=(DL_result_img.shape[::-1]), interpolation=cv2.INTER_NEAREST)
    row=[]
    row.append(os.path.split(svs_path)[-1])
    print(np.unique(mask))
    for classid in sorted(sort_index):
        classid = classid+1
        DL_result_img2 = deepcopy(DL_result_img)
        DL_result_img2[mask!=classid]=0
        # fig2,ax2=plt.subplots(1,2)
        # ax2[0].imshow((DL_result_img2==classid)*1)
        # ax2[1].imshow((mask==classid)*1)
        a=(np.sum(np.sum(((DL_result_img2==classid)*1),axis=1)/1)*1)
        b=(np.sum(np.sum(((mask==classid)*1),axis=1)/1)*1)
        bb=(np.sum(np.sum(((mask>0)*1),axis=1)/1)*1)
        print(a,b,bb)
        row.append(np.around(a/b,decimals=3))
    accuracies.loc[idx] = row

    # fig2,ax2=plt.subplots(1,2)
    # fig2.set_figheight(12)
    # fig2.set_figwidth(24)
    # ax2[0].imshow((mask>0)*255,cmap='gray')
    # ax2[1].imshow(DL_result_img,cmap='gray')
    # ax2[1].set_title('accuracy = '+str(a/b))
    # fig2.savefig(os.path.join(DL_src,'svg/manual_vs_deeplab_collagen_within_manual_{:d}.svg'.format(idx)),format='svg')
    # plt.close(fig2)

accuracies.to_csv(os.path.join(DL_src,'multiclass_accuracies.csv'),index=False)

readxml took 0.83 sec
mask generation: 1.35 sec elapsed
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
40030.0 53219.0 4493826.0
110392.0 160491.0 4493826.0
18167.0 21532.0 4493826.0
88379.0 93270.0 4493826.0
18041.0 25708.0 4493826.0
124149.0 126239.0 4493826.0
22277.0 28705.0 4493826.0
5418.0 5720.0 4493826.0
30984.0 36292.0 4493826.0
1875407.0 2023937.0 4493826.0
893969.0 1170226.0 4493826.0
748385.0 748487.0 4493826.0
readxml took 0.15 sec
mask generation: 0.79 sec elapsed
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
16504.0 20820.0 1758893.0
35505.0 38254.0 1758893.0
2334.0 3436.0 1758893.0
12082.0 12279.0 1758893.0
7797.0 24085.0 1758893.0
65981.0 68383.0 1758893.0
5912.0 8819.0 1758893.0
13524.0 14861.0 1758893.0
4012.0 5777.0 1758893.0
1106546.0 1194991.0 1758893.0
213956.0 231583.0 1758893.0
135549.0 135605.0 1758893.0
readxml took 0.11 sec
mask generation: 0.60 sec elapsed
[ 0.  1.  2.  3.  4.  5.  7.  8.  9. 10. 11. 12.]
34601.0 40247.0 1212746.0
53757.0 7997

C:\ProgramData\Anaconda3\envs\TF2Shared\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars
